In [10]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    # Regression
    return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [3]:
path = "./data/"

In [4]:
filename_read = os.path.join(path,"Train_Solar.csv")
filename_write = os.path.join(path,"Train_Solar_R5_model.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

In [5]:
# Shuffle
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)

In [6]:
df.head(1)

,datetime,date,time,I,T,UV,WS,RH,P
0,22/2/2018 6:27,22/2/2018,6:27:00,0.0,27.1,0.0,6.1,69.0,0.0


In [7]:
# Preprocess
datetime_sr = df['datetime']
date_sr = df['date']
time_sr = df['time']
df.drop('datetime',1,inplace=True)
df.drop('date',1,inplace=True)
df.drop('time',1,inplace=True)
missing_median(df, 'I')
missing_median(df, 'T')
missing_median(df, 'UV')
missing_median(df, 'WS')
missing_median(df, 'RH')
missing_median(df, 'P')

In [8]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(df)
y_noano = clf.predict(df)
y_noano = pd.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

df = df.iloc[y_noano[y_noano['Top'] == 1].index.values]
df.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", df.shape[0])

Number of Outliers: 13159
Number of rows without outliers: 118427


In [11]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(df.columns)

mat_train = np.matrix(df)

prepro = MinMaxScaler()
prepro.fit(mat_train)

df = pd.DataFrame(prepro.transform(mat_train),columns = col_train)

In [12]:
# Encode to a 2D matrix for training
x,y = to_xy(df,'P')

In [13]:
print(x[0:5])

[[0.         0.34817815 0.         0.19303797 0.8668172 ]
 [0.3666565  0.51012146 0.3364486  0.05696202 0.79458237]
 [0.28253582 0.4777328  0.1588785  0.         0.83220464]
 [0.26333436 0.58299595 0.2897196  0.26265824 0.8058691 ]
 [0.27339226 0.4979757  0.28037384 0.03164557 0.9007524 ]]


In [14]:
print(y[0:5])

[[0.        ]
 [0.3237705 ]
 [0.24385247]
 [0.22745901]
 [0.24640983]]


In [15]:
# Cross-Validate
kf = KFold(5)

In [16]:
oos_y = []
oos_pred = []
fold = 0

##### for train, test in kf.split(x):
for train, test in kf.split(x):
    fold+=1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(25,activation='relu'))
    model.add(Dense(1))
    
    
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    
    monitor = EarlyStopping(monitor='val_loss',
                            min_delta=1e-3,
                            patience=5,
                            verbose=1,
                            mode='auto')
    
    model.fit(x_train,
              y_train,
              validation_data=(x_test,y_test)
              ,callbacks=[monitor],
              verbose=0,
              epochs=1000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)        

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))

Fold #1
Epoch 00006: early stopping
Fold score (RMSE): 0.05983639881014824
Fold #2
Epoch 00006: early stopping
Fold score (RMSE): 0.059139642864465714
Fold #3
Epoch 00006: early stopping
Fold score (RMSE): 0.05952302739024162
Fold #4
Epoch 00006: early stopping
Fold score (RMSE): 0.06094333902001381
Fold #5
Epoch 00006: early stopping
Fold score (RMSE): 0.05960319936275482


In [17]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print("Final, out of sample score (RMSE): {}".format(score))   

Final, out of sample score (RMSE): 0.05981222540140152


In [18]:
# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
oosDF.to_csv(filename_write,index=False)

In [19]:
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(path,"R5_model.h5"))

In [20]:
# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(path,"R5_model.json"), "w") as json_file:
    json_file.write(model_json)

In [21]:
# save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(path,"R5_model.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)